In [ ]:
# path_to_predictions = "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/ALL_clinical_trajectories_dictionary_rules_of_thumb_yearly_2023-01-31.pkl"
path_to_predictions = "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/final_predictions/ALL_clinical_trajectories_dictionary_rules_of_thumb_yearly_2023-07-11.pkl"
output_path = "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/temporal_model/data"
general_information = "/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/input_data/General_information_20-07-2023.xlsx"



In [ ]:
#Kernel: simplemodels3
#Python version on web portal: 3.9.5
from __future__ import absolute_import, division, print_function
# import seaborn as sns; sns.set()
import numpy as np; np.random.seed(0)
# from matplotlib import pyplot as plt 

import xlsxwriter
import pandas as pd
import numpy as np
import pickle
import os
from sklearn import model_selection 
from collections import Counter
from skmultilearn.model_selection.measures import get_combination_wise_output_matrix
from natsort import natsorted
import sys
sys.path.insert(0, '/home/jupyter-n.mekkes@gmail.com-f6d87/clinical_history/scripts')
from helper_functions import table_selector

In [ ]:
if not os.path.exists(output_path):
    print('Creating output folder....')
    os.makedirs(output_path)

In [ ]:
with open(path_to_predictions,"rb") as file:
    predictions_pickle = pickle.load(file)

d = []
for i,j in zip(predictions_pickle,predictions_pickle.values()):
    k = pd.DataFrame.from_dict(j,orient="index")
    k["DonorID"] = i
    k['Age'] = k.index
    d.append(k)
predictions_df =pd.concat(d, ignore_index=True)

print(f"there are {len(list(predictions_df['DonorID'].unique()))} unique donor IDs")
print(predictions_df.shape)
# display(predictions_df.head())

In [ ]:
general_information_df = pd.read_excel(general_information, engine='openpyxl', sheet_name="Sheet1")
donors_to_remove = list(general_information_df[general_information_df['paper diagnosis']=='exclude'].DonorID)
predictions_df = predictions_df[~predictions_df['DonorID'].isin(donors_to_remove)]
print(f"there are {len(list(predictions_df['DonorID'].unique()))} unique donor IDs")
print(len(donors_to_remove))
predictions_df['neuropathological_diagnosis'] = predictions_df['DonorID'].map(general_information_df.set_index('DonorID')['paper diagnosis'])
display(predictions_df.head())
print(sorted(predictions_df['neuropathological_diagnosis'].unique()))
print(f"there are {len(list(predictions_df['DonorID'].unique()))} unique donor IDs")

In [ ]:
trajectories = predictions_df.copy()
# trajectories = trajectories.drop(['Year'], axis=1)
unique_diagnoses = trajectories[['DonorID', 'neuropathological_diagnosis']].drop_duplicates()
display(unique_diagnoses['neuropathological_diagnosis'].value_counts().head(20))
display(trajectories.head())

### remove donors before 1997

In [ ]:
trajectories['file_year'] = trajectories['DonorID'].str.extract(r'NBB (\d{4})-\d{3}', expand=False)
trajectories['file_year'] = pd.to_numeric(trajectories['file_year'])
trajectories = trajectories[trajectories['file_year'] >= 1997]
display(trajectories.head())

In [ ]:
unique_diagnoses = trajectories[['DonorID', 'neuropathological_diagnosis']].drop_duplicates()
unique_diagnoses['neuropathological_diagnosis'].value_counts().head(20)


In [ ]:
m = 1 ## obs
n = 5 ## years ## we also use this one to select for how many total observations 
## 5 means with flattened, 6 means without
remove3 = True
observation = False
unique = False
eighties = False
remove_diagnoses = ['DLB','ATAXIA']

In [ ]:
non_attribute_columns = ['DonorID','Age','neuropathological_diagnosis','age_at_death','Year','sex','file_year'] #'death_year_based_on_DonorID'


In [ ]:
trajectories = trajectories.sort_values(['DonorID', 'Age'],
              ascending = [True, True])

### Select only table 1 cases

In [ ]:
selected_diagnoses, ordered_diagnoses = table_selector('table1_p', trajectories)
print('After selecting for {}, we have {} sentences belonging to {} donors'.format(selected_diagnoses['neuropathological_diagnosis'].unique(),
                                                                                    selected_diagnoses.shape[0],
                                                                                    selected_diagnoses['DonorID'].nunique()) )



### remove/add/merge some diagnoses

In [ ]:
## PDD into PD
selected_diagnoses['neuropathological_diagnosis'] = selected_diagnoses['neuropathological_diagnosis'].replace('PDD', 'PD')
## add D,DLB
selected_diagnoses = pd.concat([selected_diagnoses, trajectories[trajectories['neuropathological_diagnosis'] == 'AD,DLB']], ignore_index=True)
# remove psych
selected_diagnoses = selected_diagnoses[selected_diagnoses['neuropathological_diagnosis'] != 'MDD']
selected_diagnoses = selected_diagnoses[selected_diagnoses['neuropathological_diagnosis'] != 'BP']
selected_diagnoses = selected_diagnoses[selected_diagnoses['neuropathological_diagnosis'] != 'SCZ']
display(selected_diagnoses.head())
unique_diagnoses = selected_diagnoses[['DonorID', 'neuropathological_diagnosis']].drop_duplicates()
unique_diagnoses['neuropathological_diagnosis'].value_counts().head(30)

### remove observations without yearly information

In [ ]:
selected_diagnoses = selected_diagnoses[selected_diagnoses.Age >= 0]
selected_diagnoses = selected_diagnoses.reset_index(drop=True)
unique_diagnoses = selected_diagnoses[['DonorID', 'neuropathological_diagnosis']].drop_duplicates()
unique_diagnoses['neuropathological_diagnosis'].value_counts().head(30)

In [ ]:
selected_diagnoses = selected_diagnoses.sort_values(['DonorID', 'Age'],
              ascending = [True, True])
selected_diagnoses.head()

#### DONORS with a diagnosis need at least n years with at least m observations


In [ ]:
## how many observations has each donor?
data = selected_diagnoses.copy()
# display(data.groupby('DonorID')['Age'].nunique())

## df showing number of observations
data2 = data.drop(columns=['age_at_death','sex','Age','file_year'])
data2 = data2.groupby(['DonorID','neuropathological_diagnosis']).sum()
data2 = pd.DataFrame(data2.sum(axis=1),columns=['count'])
data2 = data2.reset_index()  
data2 = data2.set_index('DonorID')
data2['uniqueage'] = data.groupby('DonorID')['Age'].nunique()
display(data2)

## con are the exception, they are allowed to have little data
data3 = data2[data2['neuropathological_diagnosis'] != 'CON']
donors_not_enough_data = data3.index[data3['count'] < 5].tolist()

print(donors_not_enough_data)
print(len(donors_not_enough_data))
data = data[~data['DonorID'].isin(donors_not_enough_data)]
data = data.reset_index(drop=True)

unique_diagnoses = data[['DonorID', 'neuropathological_diagnosis']].drop_duplicates()
unique_diagnoses['neuropathological_diagnosis'].value_counts().head(30)


## for figure 5

In [ ]:
print(len(data['DonorID'].unique()))

### creating data for model

#### INPUT
array of arrays. each array is for a single donor, consisting of shape time x attribute

In [ ]:
inp = data.copy()
inp = inp.drop(['neuropathological_diagnosis','file_year','age_at_death'],axis=1)
inp['sex'] = inp['sex'].map({'F': 1, 'M': 0}).astype(int)
inp['Age']  = inp['Age'].astype(int)
def sum_except_donors(df):
    return df.iloc[:, ].sum()

inp = inp.sort_values(['DonorID', 'Age'],
              ascending = [True, True])

inp_with_nan = inp.copy()
inp_with_nan = inp_with_nan.reset_index(drop=True)
display(inp_with_nan)
final_input = inp_with_nan.set_index('DonorID').groupby('DonorID').apply(pd.DataFrame.to_numpy).to_numpy()
print(final_input)

In [ ]:
print(final_input.shape)
print(final_input[0].shape)

#### LABEL TASKNAME

array in the shape samples X diagnosis

In [ ]:
np.set_printoptions(threshold=30)
lt = data[['DonorID','neuropathological_diagnosis']].copy()
# lt = lt[~lt['DonorID'].isin(weirds)]
lt = lt.drop_duplicates().reset_index(drop=True)

display(lt)

donorcount = len(lt['DonorID'])
print(donorcount)
print(lt['neuropathological_diagnosis'].value_counts())
print(lt.neuropathological_diagnosis.unique())

one_hot = pd.get_dummies(lt.neuropathological_diagnosis)

# Define the ordered list
wanted = ['CON', 'AD', 'PD', 'VD', 'FTD','DLB','AD,DLB','ATAXIA', 'MND', 'PSP', 'MS','MSA'] #'AD,DLB' 'DLB,SICC',

# Get the current columns of the dataframe
current_cols = list(one_hot.columns)

# Create a new list of columns in the order of the 'wanted' list
new_cols = [col for col in wanted if col in current_cols]

# Reorder the columns of the dataframe using the new list of columns
one_hot = one_hot.reindex(columns=new_cols)
display(one_hot)
final_label_taskname = one_hot.to_numpy()
display(final_label_taskname)

#### MASKING
Masking is a datatype of the same shape as input. it is used to indicate which data is present, and which data is absent. If a row of data would be [nan, nan, 0, 0, 3.14, 10], the masking row would be [0,0,1,1,1,1]. In our case, we have options:
- every value larger than 1 to 1, every zero to zero. because a zero can mean that the symptom is present, it is just not written down that year?
- every value to 1. 

In [ ]:
final_input

In [ ]:
import copy
final_masking = copy.deepcopy(final_input)

In [ ]:
for k in range(len(final_masking)):
    final_masking[k][final_masking[k] >= 0] = 1    

print(final_masking)

#### TIMESTAMP
Timestamp is another array of arrays. There is an array for every donor, that consists of the timepoints that are known for that donor. e.g. if donor1 has information from age 34, 61, and 62, then his timestamp would be [34,61,62].

In [ ]:
timestamp_df = inp[['DonorID','Age']].copy()
timestamp_df
final_timestamp = timestamp_df.set_index('DonorID').groupby('DonorID').apply(pd.DataFrame.to_numpy).to_numpy()
final_timestamp

### SPLITTING BALANCED

In [ ]:
## This function takes the created train and test data as input
## it returns a measure of how similar train is to test
## it also shows an overview of the number of cases per attribute
## and a corrected version of this overview
def split_vis(x_train,x_test,y_train, y_test,train_val_size,test_size):
    """
    something
    """
    counts = {}
    counts["train_counts"] = Counter(str(combination) for row in get_combination_wise_output_matrix(
        y_train, order=1) for combination in row)
    counts["test_counts"] = Counter(str(combination) for row in get_combination_wise_output_matrix(
        y_test, order=1) for combination in row)    

    # view distributions
    multi_split_dist = pd.DataFrame({
        "for_train_and_val": counts["train_counts"],
        "test": counts["test_counts"]
    }).T.fillna(0)
    multi_split_dist = multi_split_dist.reindex(natsorted(multi_split_dist.columns), axis=1)
#     multi_split_dist.columns = labels
    
    for k in counts["test_counts"].keys():
        counts["test_counts"][k] = int(counts["test_counts"][k] * (train_val_size/test_size))
        
    # View size corrected distributions
    multi_split_dist_corr = pd.DataFrame({
        "for_train_and_val": counts["train_counts"],
        "test": counts["test_counts"]
    }).T.fillna(0)
    multi_split_dist_corr =multi_split_dist_corr.reindex(natsorted(multi_split_dist_corr.columns), axis=1)
#     multi_split_dist_corr.columns = labels
    
    print(f"train: {len(x_train)} ({len(x_train)/(len(x_train)+len(x_test)):.2f})\n"
          f"test: {len(x_test)} ({len(x_test)/(len(x_train)+len(x_test)):.2f})")
    dist_split = np.mean(np.std(multi_split_dist_corr.to_numpy(), axis=0))
    
    return dist_split,multi_split_dist,multi_split_dist_corr

## for figure 5, counts for split are in here

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

if eighties == True:
    n_split = 10 # 5 for 60-20-20, 10 for 80-10-10
    n_split_in = 9 # 4 for 60-20-20, 9 for 80-10-10
elif eighties == False:
    n_split = 5 # 5 for 60-20-20, 10 for 80-10-10
    n_split_in = 4 # 4 for 60-20-20, 9 for 80-10-10

fold_taskname = np.empty(shape=(5, 3), dtype=object)

# X = np.array([0, 2, 1, 1,0,2,0, 2, 1, 1,0,2,0, 2, 1, 1,0,2])
# y = np.array([0, 2, 1, 1,0,2,0, 2, 1, 1,0,2,0, 2, 1, 1,0,2])
X = np.array(lt['neuropathological_diagnosis'].values)
y = np.array(lt['neuropathological_diagnosis'].values)
print(y)
print(y.shape)


## SET UP SPLIT BETWEEN TEST AND TRAIN/VAL
skf = StratifiedKFold(n_splits=n_split, random_state=1, shuffle=True)
skf.get_n_splits(X, y)
print(skf)
j = 0
for train_val_index, test_index in skf.split(X, y):
#     print("TRAIN+VAL:", train_val_index, "TEST:", test_index)
    ## USE THE GENERATED INDICES TO SELECT DIAGNOSES
    q_train_val, q_test = X[train_val_index], X[test_index]
    r_train_val, r_test = y[train_val_index], y[test_index]
#     print('test:' , test_index)
#     print('trainval: ', train_val_index)
    skf2 = StratifiedKFold(n_splits=n_split_in, random_state=2, shuffle=True)
    skf2.get_n_splits(X, y)
    print(skf2)
    
    ## WITHIN EACH FOLD, SPLIT TRAIN/VAL INTO TRAIN AND VAL (ONLY NEEDED ONCE!)
    i = 0
    ## example: 
    ## [0,1,2,3,4,5,6,7,8,9]  full data ['a','b','c','d','e','f','g','h','i','j']
    ## [0,2,3,5,7,9] indices selected for train/val ['a','c','d','f','h','j']
    ## [0,1,2,3,4,5]
    ## [0,2,3,9] indices points selected for train ['a','c','d','j']
    for train_index, val_index in skf2.split(q_train_val, r_train_val):
        print(i)
        if i == 0:
            ## USE THE GENERATED INDICES TO CREATE NEW INDICES THAT WORK ON THE FULL DATA
            true_train = train_val_index[train_index]
            true_val = train_val_index[val_index]
            
            ## PRINT THE INDICES
            print("TRAIN:", true_train, "\nVAL:", true_val, "\nTEST:", test_index)
            q_train, q_val, q_test = X[true_train], X[true_val],X[test_index]
            r_train, r_val, q_test = y[true_train], y[true_val],y[test_index] 
            
            #print('trainval: ',train_val_index,train_val_index.shape )
            #print('train: ',train_index, train_index.shape)
            print(f"train: {len(q_train)} ({len(q_train)/(len(q_train)+len(q_test)+len(q_val)):.2f})\n"
                  f"val: {len(q_val)} ({len(q_val)/(len(q_train)+len(q_test)+len(q_val)):.2f})\n"
                  f"test: {len(q_test)} ({len(q_test)/(len(q_train)+len(q_test)+len(q_val)):.2f})")
            
            ## SAVE INTO NUMPY ARRAY
            fold_taskname[j][0] = np.asarray(true_train)
            fold_taskname[j][1] = np.asarray(true_val)
            fold_taskname[j][2] = np.asarray(test_index)
            
            ## FOR VISUALIZING COUNTS PER DIAGNOSIS PER FOLD
            ## TRAINING
            foo, bar = np.unique(q_train, return_counts=True)
            my_dict = dict(zip(foo, bar))
            df = pd.DataFrame(list(my_dict.items()),columns = ['diagnosis','train'])
            
            ## VALIDATION
            foo, bar = np.unique(q_val, return_counts=True)
            my_dict = dict(zip(foo, bar))
            df1 = pd.DataFrame(list(my_dict.items()),columns = ['diagnosis2','val'])
            
            ## TEST
            foo, bar = np.unique(q_test, return_counts=True)
            my_dict = dict(zip(foo, bar))
            df2 = pd.DataFrame(list(my_dict.items()),columns = ['diagnosis3','test'])
            
            ## COMBINE ALL THREE
            df3 = pd.concat([df,df1, df2], ignore_index=True,axis=1)
            df3.columns = ['diagnosis','train','diagnosis2','val','diagnosis3','test']
            df3 = df3.drop(['diagnosis2','diagnosis3'], axis=1)
            display(df3)
            print(df3['diagnosis'])
#         elif i > 0:
#             print('finished fold {}, exiting...'.format(i))
            break
        i = i +1
    j = j + 1
    print('---------')
print(fold_taskname)

In [ ]:
# fold_taskname

In [ ]:
# fold_taskname[0,0]

In [ ]:
# final_input

In [ ]:
# final_input[fold_taskname[0][0]]

In [ ]:
#np.set_printoptions(threshold=np.inf)
n_dim = 86#83
mean_taskname = np.zeros((5, 1, n_dim)) * np.nan
std_taskname = np.zeros((5, 1, n_dim)) * np.nan
for i_split in range(5):
    ## fold_taskname[i_split][0] selecteert de indexen van de training donors van elke fold
    ## final_input[fold_taskname[i_split][0]] selecteerd de training data van deze donoren
    ## de concatenate step combineer het, dus x_tr is training data per fold
    x_tr = np.concatenate(final_input[fold_taskname[i_split][0]], axis=0)
    display(x_tr)
    ## mean taskname contains the mean of each training column. eg. for the first fold, the average age is 75
    mean_taskname[i_split][0] = np.nanmean(x_tr, axis=0)
    ## std taskname contains the std of each training column. eg. for the first fold, the average age is 12
    std_taskname[i_split][0] = np.nanstd(x_tr, axis=0)
    
print(mean_taskname[0][0])
print(std_taskname[0][0])
mean_taskname

### SAVING

In [ ]:
if eighties == True:
    prefix = '80_'
elif eighties == False:
    prefix = '60_'
    
if remove3 == False and observation == False and unique == False:
    savespace = '{}clinical_history_{}_years_{}_observations'.format(prefix, str(n),str(m))

elif remove3 == False and observation == True and unique == False:
    savespace = '{}clinical_history_{}_observations'.format(prefix,str(n))

elif remove3 == False and observation == True and unique == True:
    savespace = '{}clinical_history_{}_unique_observations'.format(prefix,str(n))
                                                          
        
elif remove3 == True and observation == False and unique == False:
    savespace = '{}clinical_history_{}_years_{}_observations_subset'.format(prefix,str(n),str(m))

elif remove3 == True and observation == True and unique == False:
    savespace = '{}clinical_history_{}_observations_subset'.format(prefix,str(n))
    
elif remove3 == True and observation == True and unique == True:
    savespace = '{}clinical_history_{}_unique_observations_subset'.format(prefix,str(n))
    

print(savespace)
os.makedirs(os.path.join(output_path,  savespace),
            exist_ok=True)
np.savez(os.path.join(output_path, savespace, 'data.npz'),
         input=final_input, masking=final_masking, timestamp=final_timestamp, label_taskname=final_label_taskname)
np.savez(os.path.join(output_path, savespace, 'fold.npz'),
         fold_taskname=fold_taskname, mean_taskname=mean_taskname, std_taskname=std_taskname)